In [ ]:
!pip install sentence_transformers

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import pickle

from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/Difficulty tagger/diff_dataset_new.csv' #loading the csv file
data = pd.read_csv(file_path)

#handling missing values in textual columns
data['Body'] = data['Body'].fillna('')
data['Title'] = data['Title'].fillna('')
data['Tags'] = data['Tags'].fillna('')

numerical_cols = [
    'User_Reputation', 'Bronze_badge', 'Gold_badge', 'Silver_badge',
    'User Id', 'Accepted Answer ID', 'View Count',
    'Answer Count', 'Score', 'Interval from first', 'Interval from accepted',
    'Total count urls and imgs', 'LOC', 'Question_Length'
]

# Impute missing values for numerical features
imputer = SimpleImputer(strategy='mean')
data[numerical_cols] = imputer.fit_transform(data[numerical_cols])

# Normalize numerical features
scaler = StandardScaler()
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

# Loading different models for generating text embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
# model = SentenceTransformer("BAAI/bge-m3")
# model = SentenceTransformer("flax-sentence-embeddings/stackoverflow_mpnet-base")
# model = SentenceTransformer("hkunlp/instructor-xl")

# Encoding all the text data and concatenating their embeddings
title_embeddings = model.encode(data['Title'].tolist(), show_progress_bar=True)
body_embeddings = model.encode(data['Body'].tolist(), show_progress_bar=True)
tags_embeddings = model.encode(data['Tags'].tolist(), show_progress_bar=True)

# Combining them into a single array
text_embeddings = np.hstack((title_embeddings, body_embeddings, tags_embeddings))

# Now combining text embeddings with numerical features
numerical_features = data[numerical_cols]
combined_features = np.hstack((text_embeddings, numerical_features))

# label encoding -
label_mapping = {'Basic': 0, 'Intermediate': 1, 'Advanced': 2}
labels = data['Label'].map(label_mapping).values

# data splitting
X_train, X_test, y_train, y_test = train_test_split(combined_features, labels, test_size=0.2, random_state=42, stratify=labels)

# Applying SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

#Applying Grid search for cross validation and hyperparameter tuning
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score

param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'bootstrap': [True, False]
}

rf_classifier = RandomForestClassifier(random_state=42)
scorer = make_scorer(accuracy_score)
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, scoring=scorer, cv=5, n_jobs=-1)
grid_search.fit(X_train_resampled, y_train_resampled)
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"Best Parameters: {best_params}")
print(f"Best Score: {best_score}")

#Fitting with best parameters
clf = RandomForestClassifier(random_state=42, **best_params)
clf.fit(X_train_resampled, y_train_resampled)

# Evaluation of model -
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=label_mapping.keys()))

# Saving the trained model and other relevant functions for later use in the extension backend
with open('trained_classifier_if.pkl', 'wb') as f:
    pickle.dump(clf, f)
print("Trained classifier saved as 'trained_classifier_if.pkl'")

with open('imputer_if.pkl', 'wb') as f:
    pickle.dump(imputer, f)
print("Trained imputer saved as 'imputer_if.pkl'")

with open('scaler_if.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("Trained scaler saved as 'scaler_if.pkl'")
